In [1]:
import seaborn as sns
import pandas as pd

In [2]:
df = sns.load_dataset("tips")
df.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


Il existe en pandas 4 manières (plus quelques cas spéciaux) de combiner des dataframes:
- Merging
- Joining
- Concatenating
- Appending

`merge` et `join` font pratiquent la même chose, et `concatenate` et `append` sont également redondant.

(`append` [vient juste d'être deprecated](https://pandas.pydata.org/docs/whatsnew/v1.4.0.html#deprecated-dataframe-append-and-series-append) )


On va donc juste apprendre `merge` et `concatenate`.

### Merge

Fonctionne dans l'esprit très proche des requêtes SQL.

In [3]:
tips_bill = df.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()
tips_tip = df.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()

del tips_bill['tip']
del tips_tip['total_bill']

In [4]:
tips_bill

total_bill
sex    smoker            
Male   Yes        1337.07
       No         1919.75
Female Yes         593.27
       No          977.68

In [5]:
tips_tip

tip
sex    smoker        
Male   Yes     183.07
       No      302.00
Female Yes      96.74
       No      149.77

In [7]:
pd.merge?

Signature:
pd.merge(
    left,
    right,
    how: str = 'inner',
    on=None,
    left_on=None,
    right_on=None,
    left_index: bool = False,
    right_index: bool = False,
    sort: bool = False,
    suffixes=('_x', '_y'),
    copy: bool = True,
    indicator: bool = False,
    validate=None,
) -> 'DataFrame'
Docstring:
Merge DataFrame or named Series objects with a database-style join.

The join is done on columns or indexes. If joining columns on
columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
on indexes or indexes on a column or columns, the index will be passed on.
When performing a cross merge, no column specifications to merge on are
allowed.

Parameters
----------
left : DataFrame
right : DataFrame or named Series
    Object to merge with.
how : {'left', 'right', 'outer', 'inner', 'cross'}, default 'inner'
    Type of merge to be performed.

    * left: use only keys from left frame, similar to a SQL left outer join;
      preserve key order.


In [6]:
# we can merge on the indexes
pd.merge(tips_bill, tips_tip, 
         right_index=True, left_index=True)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [7]:
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index(),
    on=['sex', 'smoker']
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


⚠️ si on ne donne pas de clé, ça merge par défault directement sur l'index.

In [13]:
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index()
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [17]:
tips_tip

tip
sex    smoker        
Male   Yes     183.07
       No      302.00
Female Yes      96.74
       No      149.77

In [14]:
pd.merge(
    tips_bill.reset_index(), 
    tips_tip,
    left_on=['sex', 'smoker'],
    right_index=True
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


### Left, inner and outer

![](images/join_venn_diagram.jpeg)

In [18]:
pd.merge(
    tips_bill.reset_index().tail(3), 
    tips_tip.reset_index().head(3),
    how='outer',
    indicator=True
)

,sex,smoker,total_bill,tip,_merge
0,Male,No,1919.75,302.00,both
1,Female,Yes,593.27,96.74,both
2,Female,No,977.68,NaN,left_only
3,Male,Yes,NaN,183.07,right_only


#### Concatenation

Merge permet de faire tout ce que fait concatenante, mais concatenate facilite les choses et permet d'appliquer une combinaison sur plus de 2 dataframe à la fois.

In [21]:
pd.concat([tips_bill, tips_bill, tips_tip], sort=True)

tip  total_bill
sex    smoker                    
Male   Yes        NaN     1337.07
       No         NaN     1919.75
Female Yes        NaN      593.27
       No         NaN      977.68
Male   Yes        NaN     1337.07
       No         NaN     1919.75
Female Yes        NaN      593.27
       No         NaN      977.68
Male   Yes     183.07         NaN
       No      302.00         NaN
Female Yes      96.74         NaN
       No      149.77         NaN

In [23]:
pd.concat([tips_bill, tips_bill, tips_tip], sort=False)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes            NaN  183.07
       No             NaN  302.00
Female Yes            NaN   96.74
       No             NaN  149.77

In [28]:
pd.concat([tips_bill, tips_tip], axis=1) # pd.merge(tips_bill, tips_tip, right_index=True, left_index=True)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

https://pandas.pydata.org/docs/user_guide/merging.html